In [0]:
import numpy as np
import pandas as pd

In [0]:
data = np.loadtxt('Data set for microchip quality Assurance 2019.txt', delimiter = ',')
np.random.seed(1)
np.random.shuffle(data)
x = data[:,:2]
y = data[:,2].astype(int)
y1 = data[:,2].astype(int)

normalising

In [25]:
max_x0 = np.amax(x[:,0])
min_x0 = np.amin(x[:,0])
max_x1 = np.amax(x[:,1])
min_x1 = np.amin(x[:,1])
print(max_x0, min_x0, max_x1, min_x1)
x0 = (x[:,0]-min_x0)/(max_x0-min_x0)
x1 = (x[:,1]-min_x1)/(max_x1-min_x1)
x_nor = np.vstack((x0, x1, y1)).T
print(x_nor)

1.0709 -0.83007 1.1089 -0.76974
[[0.38181034 0.93775284 0.        ]
 [0.3272645  0.50195354 1.        ]
 [0.55453269 0.82491057 0.        ]
 [0.18181244 0.94164396 0.        ]
 [0.74846526 0.01556445 0.        ]
 [0.81816125 0.64591939 1.        ]
 [0.35150476 0.83658391 1.        ]
 [0.51513701 0.10116893 0.        ]
 [0.66967916 0.76654388 1.        ]
 [0.64543891 0.84825725 0.        ]
 [0.58786304 0.98834263 0.        ]
 [0.77270551 0.88327726 0.        ]
 [0.32423447 0.77821722 1.        ]
 [0.16969231 0.89105949 0.        ]
 [0.05454584 0.36576353 0.        ]
 [0.32120444 0.40078301 1.        ]
 [0.35756482 0.2140112  0.        ]
 [1.         0.46304241 0.        ]
 [0.94240309 0.45525806 0.        ]
 [0.76058539 0.79767278 0.        ]
 [0.58786304 0.         0.        ]
 [0.41211119 0.84436614 1.        ]
 [0.22120812 0.19066452 0.        ]
 [0.         0.57587936 0.        ]
 [0.89088728 0.36576353 0.        ]
 [0.39392994 1.         0.        ]
 [0.27877873 0.42412969 1.      

In [0]:
x_nor = x_nor[x_nor[:,0] != 0]
x_nor = x_nor[x_nor[:,1] != 0]

Applying Box muller

In [0]:
z0 = np.sqrt(-2*np.log(x_nor[:, 0]))*np.cos(2*np.pi*x_nor[:,1])
z1 = np.sqrt(-2*np.log(x_nor[:, 0]))*np.sin(2*np.pi*x_nor[:,1])
z = np.vstack((z0, z1,x_nor[:,2])).T

In [0]:
train_data = z[0:int(0.7*len(z))]
test_data = z[int(0.7*len(z)):] 

In [0]:
train_0 = train_data[train_data[:,2] == 0]
train_1 = train_data[train_data[:,2] == 1]
mean_0 = np.sum(train_0[:,0:2], axis = 0)/len(train_0)
mean_1 = np.sum(train_1[:,0:2], axis = 0)/len(train_1)

In [30]:
print(mean_0, mean_1)

[0.25142824 0.11980947] [-0.32913003 -0.24003045]


In [0]:
sigma = np.zeros((2,2))
for xi in train_0[:,0:2]:
  sigma += np.matmul((xi.reshape(1,2)-mean_0).T,(xi.reshape(1,2) - mean_0))
# print(sigma)
for xi in train_1[:, 0:2]:
  sigma += np.matmul((xi.reshape(1,2)-mean_1).T,(xi.reshape(1,2) - mean_1))


In [36]:
print(sigma)
sigma = sigma / len(train_data)
print(sigma)

[[ 53.99606424 -11.6921266 ]
 [-11.6921266   89.84993332]]
[[ 0.66661808 -0.14434724]
 [-0.14434724  1.10925844]]


In [0]:
phi = len(train_1)/len(train_data)
prior_y1 = phi
prior_y0 = (1-phi) 

In [0]:
y_pred = np.zeros((len(test_data),1))
det_cov = np.linalg.det(sigma)
n = len(test_data.T) - 1
i = 0
inv_sigma = np.linalg.inv(sigma)
for x in test_data[:,0:2]:
  class_0 = -0.5*np.matmul(np.matmul((x.reshape(1,2)-mean_0),inv_sigma),(x.reshape(1,2)-mean_0).T)
  class_1 = -0.5*np.matmul(np.matmul((x.reshape(1,2)-mean_1),inv_sigma),(x.reshape(1,2)-mean_1).T)
  p0 = 1/((2*np.pi)**(n/2)*np.sqrt(det_cov))*(np.exp(class_0))
  p1 = 1/((2*np.pi)**(n/2)*np.sqrt(det_cov))*(np.exp(class_1))
  if (prior_y0*p0 > prior_y1*p1):
    y_pred[i,0] = 0
  else:
    y_pred[i,0] = 1
  i += 1

In [39]:
count = 0
for i in range(len(y_pred)):
  if(y_pred[i,0] == test_data[i,2]):
    count += 1

print("accuracy = ",count/len(y_pred))

accuracy =  0.5428571428571428


Without Box_Muller transform

In [40]:
train1_data = data[0:int(0.7*len(z))]
test1_data = data[int(0.7*len(z)):]
train1_0 = train1_data[train1_data[:,2] == 0]
train1_1 = train1_data[train1_data[:,2] == 1]
mean1_0 = np.sum(train1_0[:,0:2], axis = 0)/len(train1_0)
mean1_1 = np.sum(train1_1[:,0:2], axis = 0)/len(train1_1) 
sigma1 = np.zeros((2,2))
for xi in train1_0[:,0:2]:
  sigma1 += np.matmul((xi.reshape(1,2)-mean1_0).T,(xi.reshape(1,2) - mean1_0))
# print(sigma)
for xi in train1_1[:, 0:2]:
  sigma1 += np.matmul((xi.reshape(1,2)-mean1_1).T,(xi.reshape(1,2) - mean1_1))

sigma1 = sigma1 / len(train1_data)
print(sigma1)

[[ 0.24308754 -0.02218281]
 [-0.02218281  0.26855462]]


In [0]:
phi1 = len(train1_1)/len(train1_data)
prior1_y1 = phi1
prior1_y0 = (1-phi1) 
y1_pred = np.zeros((len(test1_data),1))
det1_cov = np.linalg.det(sigma1)
n1 = len(test1_data.T) - 1
i = 0
inv_sigma1 = np.linalg.inv(sigma1)
for x in test1_data[:,0:2]:
  class_0 = -0.5*np.matmul(np.matmul((x.reshape(1,2)-mean1_0),inv_sigma1),(x.reshape(1,2)-mean1_0).T)
  class_1 = -0.5*np.matmul(np.matmul((x.reshape(1,2)-mean1_1),inv_sigma1),(x.reshape(1,2)-mean1_1).T)
  p0 = 1/((2*np.pi)**(n/2)*np.sqrt(det1_cov))*(np.exp(class_0))
  p1 = 1/((2*np.pi)**(n/2)*np.sqrt(det1_cov))*(np.exp(class_1))
  if (prior1_y0*p0 > prior1_y1*p1):
    y1_pred[i,0] = 0
  else:
    y1_pred[i,0] = 1
  i += 1

In [42]:
count1 = 0
for i in range(len(y1_pred)):
  if(y1_pred[i,0] == test1_data[i,2]):
    count1 += 1

print("accuracy without B-M transform = ",count1/len(y1_pred))

accuracy without B-M transform =  0.40540540540540543
